In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from pprint import pprint
import pandas as pd
import simuran
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
from simuran.loaders.one_loader import OneAlyxLoader
from simuran.loaders.nwb_loader import NWBLoader

In [ ]:
ALLEN_CACHE_DIR = Path(r"D:\example-data\allen-data")
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.4.0.json"
IBL_CACHE_DIR = Path(r"D:\example-data\ibl-data")

In [ ]:
allen_loader = AllenVisualBehaviorLoader(
    cache_directory=ALLEN_CACHE_DIR, manifest=ALLEN_MANIFEST
)
allen_loader.create_s3_cache()
one_loader = OneAlyxLoader.from_cache(IBL_CACHE_DIR)

allen_sessions = allen_loader.cache.get_ecephys_session_table()
allen_sessions = allen_sessions.dropna(subset=["file_id"])
one_sessions = [s for s in one_loader.sessions]
one_df = pd.DataFrame([s for s in one_sessions])

In [ ]:
allen_recording_container = simuran.RecordingContainer.from_table(allen_sessions, allen_loader)
one_recording_container = simuran.RecordingContainer.from_table(one_df, one_loader)

In [ ]:
# allen_recording_container.table.to_csv("allen_recordings.csv")
# one_recording_container.table.to_csv("one_recordings.csv")

In [ ]:
id_name = allen_recording_container[0].attrs["id_name"]
example_allen_recording = allen_recording_container.find_recording_by_attribute(
    id_name, 1044385384
)
example_allen_recording.load()

In [ ]:
nwb_loader = NWBLoader()
nwb_source_file = allen_loader.path_to_nwb(example_allen_recording)
nwb_allen_recording = simuran.Recording(source_file=nwb_source_file, loader=nwb_loader)
nwb_allen_recording.load()

In [ ]:
eid = one_loader.find_eid("steinmetzlab", "NR_0017")[0][0]
example_one_recording = one_recording_container.find_recording_by_attribute(
    "session", eid
)
example_one_recording.load()


In [ ]:
def extract_useful_one(recording):
    pass

In [ ]:
def extract_useful_allen(recording):
    session = recording.data
    units = session.get_units()
    channels = session.get_channels()
    unit_channels = units.merge(channels, left_on="peak_channel_id", right_index=True)
    unit_channels.to_csv("units.csv")

In [ ]:
example_allen_recording.data.metadata

In [ ]:
print(nwb_allen_recording.data)

In [ ]:
[example_one_recording.data.keys()]